Data Preprocessing

In [1]:
#installing the natural language tool kit for normalization of the dataset
!pip install nltk

     ---------------------------------------- 42.0/42.0 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 1.5/1.5 MB 4.8 MB/s eta 0:00:00
   ---------------------------------------- 269.5/269.5 kB 8.1 MB/s eta 0:00:00


DEPRECATION: colab 1.13.5 has a non-standard dependency specifier pytz>=2011n. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of colab or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [4]:
#imporiting the required dependencies
from collections import defaultdict
import nltk
from nltk.corpus import wordnet as wn, stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
!pip install transformers

     -------------------------------------- 131.1/131.1 kB 2.6 MB/s eta 0:00:00
   ---------------------------------------- 8.5/8.5 MB 5.9 MB/s eta 0:00:00
   ---------------------------------------- 330.1/330.1 kB 6.8 MB/s eta 0:00:00
   ---------------------------------------- 269.7/269.7 kB 5.5 MB/s eta 0:00:00
   ---------------------------------------- 2.2/2.2 MB 6.1 MB/s eta 0:00:00
   ---------------------------------------- 170.9/170.9 kB 5.2 MB/s eta 0:00:00


DEPRECATION: colab 1.13.5 has a non-standard dependency specifier pytz>=2011n. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of colab or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [16]:
data_df = pd.read_csv("../dataset/dataset1.csv",sep=",",header=0,quotechar='"')

In [7]:
# max and minimum length review
max_length_review = data_df["review_text"].iloc[data_df["review_text"].str.len().idxmax()]
print(len(max_length_review))

min_length_review = data_df["review_text"].iloc[data_df["review_text"].str.len().idxmin()]
print(len(min_length_review))

523
9


In [14]:
from transformers import BertTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from collections import defaultdict
from nltk.corpus import wordnet as wn

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define NLTK resources
lemmatizer = WordNetLemmatizer()
english_stopwords = set(stopwords.words('english'))
english_stopwords = english_stopwords - {"aren't", "won't", 'nor',  "doesn't", "weren't", "shan't", 'no', 'hadn', "couldn't", "don't", 'needn', "shouldn't",  'shouldn', 'mustn', "wasn't", 'isn', "needn't", 'ain', "hadn't", "haven't",  "didn't", 'didn', 'couldn', 'weren',  "mustn't", 'hasn', 'mightn', 'wouldn', "wouldn't",  "isn't", 'not', "hasn't",  'wasn',  'aren', 'haven', "mightn't",  'doesn',  'won', 'don'}
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

def preprocess_comments_bert(text, max_length=128):
  tokenized_reviews = []
  attention_masks = []
  print(text)
  # Convert to lowercase
  review = text.lower()

  # Tokenize using NLTK word tokenizer
  tokens = word_tokenize(review)

  # Remove stopwords and perform lemmatization
  filtered_tokens = [lemmatizer.lemmatize(word, tag_map[tag[0]]) for word, tag in pos_tag(tokens)
                      if word.isalpha() and word not in english_stopwords]
  print(filtered_tokens)

  # Truncate to max_length
  filtered_tokens = filtered_tokens[:max_length]

  # Tokenize using BERT tokenizer
  encoded_dict = tokenizer.encode_plus(
                      ' '.join(filtered_tokens),  # Join tokens back into a string
                      add_special_tokens=True,    # Add '[CLS]' and '[SEP]'
                      max_length=max_length,      # Pad & truncate to max_length
                      padding=True,
                      return_attention_mask=True,  # Construct attention masks
                      return_tensors='pt',        # Return PyTorch tensors
                      truncation=True
                  )

  # Add tokenized review and its attention mask to the lists
  tokenized_reviews.append(encoded_dict['input_ids'])
  attention_masks.append(encoded_dict['attention_mask'])

  return (tokenized_reviews, attention_masks)


In [17]:
data_df["review_text"] = data_df["review_text"].apply(preprocess_comments_bert)

Make it like better with a giant pig bigger than king pig.
['make', 'like', 'well', 'giant', 'pig', 'big', 'king', 'pig']
These screens are small enough without crowding in more stuff.Tuck it in with the rest of the add ons I never use and please get it out of my face!
['screen', 'small', 'enough', 'without', 'crowd', 'rest', 'add', 'ons', 'never', 'use', 'please', 'get', 'face']
Dear Rovio; If you absolutely must continue trying to squeeze dollars from your customers this way, at least have the decency to include an opt out for those like myself who are absolutely NOT going to spend an amount equal to what I paid for the app in the first place just to have some gimmicky and expendable in game item.
['dear', 'rovio', 'absolutely', 'must', 'continue', 'try', 'squeeze', 'dollar', 'customer', 'way', 'least', 'decency', 'include', 'opt', 'like', 'absolutely', 'not', 'go', 'spend', 'amount', 'equal', 'pay', 'app', 'first', 'place', 'gimmicky', 'expendable', 'game', 'item']
App crashes when 

In [20]:
data_df["review_text"][0]

([tensor([[  101,  2191,  2066,  2092,  5016, 10369,  2502,  2332, 10369,   102]])],
 [tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])])

In [19]:
data_df

,review_text,F,BR,AU,FI,IR,A,L,LF,MN,O,PE,SC,SE,US,PO
0,"([[tensor([ 101, 2191, 2066, 2092, 5016, ...",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,"([[tensor([ 101, 3898, 2235, 2438, 2302, 4306,...",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,"([[tensor([ 101, 6203, 20996, 25500, 7078, ...",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,"([[tensor([ 101, 10439, 5823, 2047, 2373, ...",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"([[tensor([ 101, 2052, 3835, 10651, 2134, ...",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1229,"([[tensor([ 101, 2204, 10439, 4299, 2147, ...",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1230,"([[tensor([ 101, 8670, 2243, 17311, 4646, ...",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1231,"([[tensor([ 101, 6904, 2615, 2293, 102])]], [...",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1232,"([[tensor([ 101, 3291, 4030, 2770, 10651, ...",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [24]:
!mkdir output

In [25]:
#Saving the DataFrame as in serialized format to preserve the properties of the data structures(Tensor)

import pickle

#Define the file path where you want to save the serialized DataFrame
file_path = 'output/serialized_review_dataframe.pkl'

# Step 3: Save the DataFrame using pickle
with open(file_path, 'wb') as f:
    pickle.dump(data_df, f)

print("DataFrame saved as a serialized file:", file_path)


DataFrame saved as a serialized file: output/serialized_review_dataframe.pkl
